In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from torch.utils.data import DataLoader, TensorDataset
import torch

## Read and Preprocess Dataset

In [21]:
#OCR methods
from pdf2image import convert_from_path
import pytesseract
from pytesseract import Output
from PIL import Image
import cv2

pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files (x86)/Tesseract-OCR/tesseract.exe'

def get_text(resume_path):
    res = ""
    images = convert_from_path(resume_path, poppler_path = 'C:/Python311/Lib/poppler-23.11.0/Library/bin')

    for i in range(len(images)):
        text = pytesseract.image_to_string(images[i], lang='eng')
        data = pytesseract.image_to_data(images[i])
        res += text
    
    return res

import spacy
from spacy.matcher import Matcher
import re

nlp = spacy.load('en_core_web_sm')

def get_important_info(resume_text):
    doc = nlp(resume_text)
    job_titles = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
    res = set(job_titles)
    res = list(res)
    return res

In [22]:
df2 = pd.read_csv("C:/Clubs and Society/DevHack/DevHack/dataset/Resume/Resume.csv")
df2 = df2[['Resume_str', 'Category']]
df2.head(5)

,Resume_str,Category
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR
1,"HR SPECIALIST, US HR OPERATIONS ...",HR
2,HR DIRECTOR Summary Over 2...,HR
3,HR SPECIALIST Summary Dedica...,HR
4,HR MANAGER Skill Highlights ...,HR


In [23]:
df2['important_info'] = df2['Resume_str'].apply(get_important_info)
df1 = df2[['important_info', 'Category']]
df1.tail(5)

,important_info,Category
2479,"[Colorado School of Mines, the Non-Commissione...",AVIATION
2480,"[WSDOT, Hyperion, the Bored Tunnel, Oversees, ...",AVIATION
2481,"[Performed, Windows, Problem Solving, User Tra...",AVIATION
2482,"[CPR & Automated External Defibrillator, GYN C...",AVIATION
2483,[State Business Management Administration ...,AVIATION


In [20]:
# Convert data to a DataFrame
df = pd.DataFrame(df1)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Tokenize and preprocess the data
def preprocess_text(texts, max_len=512):
    tokens = []
    for text in texts:
        # Use 'encode_plus' for each text in the list
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding="max_length",
            return_attention_mask=True,
            return_tensors="pt"
        )
        tokens.append(encoding)
    return tokens

# Apply the preprocessing function to the 'important_info' column
df['tokens'] = df['important_info'].apply(preprocess_text)

# Find the maximum length of tokens or default to 0 if there are no tokens
max_len = max(max(len(token['input_ids'][0]) for token in tokens) if tokens else 0 for tokens in df['tokens'])

# Pad or truncate sequences to the maximum length
for i, tokens in enumerate(df['tokens']):
    for j, token in enumerate(tokens):
        for key in token:
            df['tokens'][i][j][key] = token[key][:max_len]

# Flatten the list of tokens for each sample
flat_tokens = [token for tokens in df['tokens'] for token in tokens]

# Create input tensors
input_ids = torch.cat([tensor['input_ids'] for tensor in flat_tokens], dim=0)
attention_masks = torch.cat([tensor['attention_mask'] for tensor in flat_tokens], dim=0)

# Repeat labels for each token
labels = torch.tensor(df['Category'].astype('category').cat.codes.values.repeat(len(flat_tokens[0])))

# Create a PyTorch DataLoader
dataset = TensorDataset(input_ids, attention_masks, labels)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

AssertionError: Size mismatch between tensors